<h1>注意</h1>
<h1>このチュートリアルは作成途中のものになります。</h1>

このノートブックではDigit Recognizeのチュートリアルを行っていきます。
なおここでは、digit_datasフォルダにデータがあると仮定して進めます。kaggleのkernelを使用している場合、データは../input/にあるので、データのパスをdigit_datasから../inputに変更してください
またこのノートブックではtensor flowがインストールされているという前提で話を進めます。お手元の環境で分析を行う場合はtensor flowをインストールしてください。

Digit Recognizerの目的は手書きの文字を認識、分類することです。このテーマで使用されているデータセットは非常に有名でMNISTというデータセットになります。<br>
ではどのように識別したらよいでしょうか<br>
ここではディープラーニングを使用して分類してみましょう。

ではまずディープラーニングについて話をしたいと思います。よく聞くワードですがディープラーニングとは果たして何なのでしょうか？<br>
ディープラーニングとは多層ニューラルネットワークのことです。しかし、こういわれてもニューラルネットワークとは？から始めないといけないと思います。<br>
ニューラルネットワークとは脳をコンピュータで再現しようというものです。ニューロンというものを組み合わせることでネットワークを作り、脳のようなものをコンピュータで再現しています。<br>
ニューラルネットワークは、まず脳の最小単位であるニューロンを再現しようというところから始まりました。その研究成果がパーセプトロンというモデルです。<br>
ニューロンはいくつかの入力と１つの出力を持っています。
![Perceptron.png](tutorial_image/Perceptron.png)
ニューロンは面白い仕組みを持っていて、各入力にある係数をかけた値の総和が一定以上になった時だけ出力を返すという仕組みを持っています。数学的に表すとn個の入力を$x_1,x_2,...,x_n$、重みを$w_1,w_2,...,w_n$、出力を返す時のある値を$a$とすると<br>
$$
\begin{equation*}
f( w_1 x_1 + w_2 x_2 + ...  + w_n x_n ) =    
    \left\{
        \begin{array}{l}
          1 (if w_1 x_1 + w_2 x_2 + ...  + w_n x_n \geq a) \\
          0 (if w_1 x_1 + w_2 x_2 + ...  + w_n x_n < a)
        \end{array}
    \right.
\end{equation*}
$$
となります。見やすいように総和記号を使用すると
$$
\begin{equation*}
    u = \sum_{i=1}^{n}w_i x_i
\end{equation*}
$$
と置くと
$$
\begin{equation*}
f(u) =    
    \left\{
        \begin{array}{l}
          1 (if u \geq a) \\
          0 (if u < a)
        \end{array}
    \right.
\end{equation*}
$$
となります。<br>
このパーセプトロンの重みの値を変化させていくことでパーセプトロンは学習していきます。ではどのように変化させていけばよいでしょうか？<br>
簡単に思いつく方法としては誤差を小さくしていく方向に重みを変化させるという方法です。では誤差を小さくしていく方向とはいったいどのような方向でしょうか？<br>
誤差が小さくなる方向とは誤差の変化率がマイナスになる方向のことというのは簡単にわかると思います。ここで、高校の数学を思い出してみてください、変化の割合率をどのように計算したでしょうか？<br>
ここで登場するのが微分です。微分は傾きを表しているというのを聞いたことがあると思います。<br>
微分とはある関数から傾きを求める操作のことを言います。微分された関数(導関数)の定義は、変数$x$の微小変化を$\Delta x$と置くと
$$
\begin{equation*}
    \frac{df(x)}{dx} = \lim_{\Delta x \to 0} \frac{f(x+\Delta x) - f(x)}{\Delta x}
\end{equation*}
$$
となります。ここで$\lim_{\Delta x \to 0}$は$\Delta x$を0にずっと近づけるということを表しています。近づけるの具体的な意味を知りたい方はぜひ$\epsilon - \delta$論法について調べてみてください。数学の奥深さ、美しさを知ることができます。<br>
変数が一つの関数であれば上の定義で問題ないでしょう、しかし問題にしているパーセプトロンは変数がいくつもあります。その場合どうしたらよいでしょうか?<br>
ここで、微分の定義に少し手を加えた偏微分というものをつかいます。偏微分は大雑把に言えば微分の対象としている変数以外を定数とみて微分することです。式で表すと<br>
$$
\begin{equation*}
    \frac{\partial f}{\partial x_{i}} = \lim_{\Delta x_i \to 0} \frac{f(x_1,x_2,...,x_i+\Delta x_i,..,x_n) - f(x_1,x_2,...,x_i,..,x_n)}{\Delta x_i}
\end{equation*}
$$
となります。
以上より重みの更新の式は、誤差の関数(誤差関数)$E$の傾きのマイナス方向に重みを更新していけばよいので、i番目の現在の重みを$w_{it}$、更新後の重みを$w_{it+1}$と置くと
$$
\begin{equation*}
    w_{it+1} = w_{it} - \frac{\partial E}{\partial w_{it}}
\end{equation*}
$$
となります。
しかし、これでは変化の割合が大きすぎることがあるので、ある正の値$\alpha$をかけて
$$
\begin{equation*}
    w_{it+1} = w_{it} - \alpha \frac{\partial E}{\partial w_{it}}
\end{equation*}
$$
としたものを更新の式にします。
パーセプトロンの誤差関数は次のようになります。
$$
\begin{equation*}
    E = max(0, -tu)
\end{equation*}
$$
これからパーセプトロンの重みの更新は
$$
\begin{eqnarray*}
    \frac{\partial E}{\partial w_{i}}& = &\frac{\partial (max(0, -tu))}{\partial w_{i}}\\
                                     & = &\frac{\partial (-t(w_1 x_1 + w_2 x_2 + ... + w_i x_i +  + w_n x_n))}{\partial w_i}\\
                                     & = &-tx_i
\end{eqnarray*}
$$
より
$$
\begin{eqnarray*}
    w_{it+1}& = &w_{it} - \alpha \frac{\partial E}{\partial w_{it}}\\
            & = &w_{it} + \alpha tx_i
\end{eqnarray*}
$$
となります。
これを用いることでパーセプトロンは学習していきます。

これらより、パーセプトロンでの私たちが設定しなければならないパラメーターは$\alpha$と$w_1,w_2,...,w_n$の初期値ということになります。

このパーセプトロンを使用すると線形分離可能な問題というものに対応できることがわかりました。線形分離可能な問題とは一本線を引いて分類可能なものを分類分けするといったものです。<br>
しかし、実際の問題は複雑な曲線で表されることが多く、そのような場合パーセプトロンは使い物になりません。ではどのようにしたらよいでしょうか。<br>
その答えになるのが多層パーセプトロンです。

多層パーセプトロンは簡単に言うと複数個パーセプトロンをつなげたものになります。<br>
![Multilayer_perceptron.png](tutorial_image/Multilayer_perceptron.png)
このようにすると何がうれしいのでしょうか？それは複雑な曲線で表されるような現象に対しても対応できるようになるためです。<br>
具体的な説明ははぶきますが、興味がある人はXOR,パーセプトロンなどのワードで検索してみてください。<br>

では早速多層パーセプトロンの説明をしたいと思います。がその前に線形代数という数学のさわりをしていきたいと思います。

線形代数学は行列やベクトルというものを取り扱うための数学です。まずはベクトルについて説明します。<br>
ベクトルはいくつかの数を一つに並べたものです。
$$
\begin{equation*}
  A = \left(
    \begin{array}{ccc}
      a_1 \\
      a_2 \\
      \vdots \\
      a_n
    \end{array}
  \right)
\end{equation*}
$$
ここで、Aはn次元のベクトルといいます。つまりベクトルがm個の数からできているとき、そのベクトルはm次元のベクトルとなります。<br>
またベクトルを作っている$a_1$や$a_n$といった数をベクトルの要素といいます。本によっては元とも言いますが、このチュートリアルでは要素で統一します。

次にベクトルの足し算、引き算を説明したいと思います。ベクトルの足し算、引き算は意外と簡単で各要素ごとに足し引きするだけです。つまり
$$
  A = \left(
    \begin{array}{ccc}
      a_1 \\
      a_2 \\
      \vdots \\
      a_n
    \end{array}
  \right),
  B = \left(
    \begin{array}{ccc}
      b_1 \\
      b_2 \\
      \vdots \\
      b_n
    \end{array}
  \right)
$$
というベクトルの足し算$A+B$は
$$
  A+B = \left(
    \begin{array}{ccc}
      a_1+b_1 \\
      a_2+b_2 \\
      \vdots \\
      a_n+b_n
    \end{array}
  \right)
$$
となり、ベクトルの引き算$B-A$は
$$
  A-B = \left(
    \begin{array}{ccc}
      a_1-b_1 \\
      a_2-b_2 \\
      \vdots \\
      a_n-b_n
    \end{array}
  \right)
$$
となります。<br>
ここで注意が必要で、ベクトルの足し算、引き算は同じ次元のベクトルでしか行えません。<br>
足し算、引き算があるならベクトルでの掛け算、割り算は?となると思います、ベクトルの掛け算は存在しますが、割り算は存在しません。<br>

ではベクトルの掛け算の一種である、内積について話そうと思います。<br>
内積はベクトルの掛け算ですが、その結果はベクトルではなく数になります。<br>
内積$A \cdot B$の定義は、上のベクトル$A,B$を使うと
$$
    \begin{equation*}
        A \cdot B = a_1 b_1 + a_2 b_2 + ... + a_n b_n
    \end{equation*}
$$
という各要素ごとの掛け算の結果を足し合わせたものになります。<br>
内積も足し算、引き算と同じく、同じ次元のベクトル同士でしか行えませんので注意してください。<br>
また、ベクトルのほかの掛け算として外積が存在しますが、ここでは説明を省きます。

次に説明するのは行列です、行列は数の集まりであるベクトルをいくつか集めたものになります。<br>
$$
\begin{equation*}
  A = \left(
    \begin{array}{cccc}
      a_{11} & a_{12} & \ldots & a_{1n} \\
      a_{21} & a_{22} & \ldots & a_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      a_{m1} & a_{m2} & \ldots & a_{mn}
    \end{array}
  \right)
\end{equation*}
$$
ここで、$A$はm行n列の行列といいます。$A$の行数はm、列数はnといいます。<br>
また、$a_{12}$のような行列を構成する数のことを、$A$の要素といい、i行j列目の要素を$A$の$(i,j)$要素といいます。<br>
ベクトルでは足し算、引き算、掛け算ができました。行列ではどうでしょうか？<br>
行列でも足し算、引き算、掛け算が行えます。最初に足し算、引き算を説明したいと思います。<br>
行列での足し算、引き算はベクトルの時のように要素ごとに足し算、引き算を行います。<br>
$$
    A = \left(
    \begin{array}{cccc}
      a_{11} & a_{12} & \ldots & a_{1n} \\
      a_{21} & a_{22} & \ldots & a_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      a_{m1} & a_{m2} & \ldots & a_{mn}
    \end{array}
    \right),
    B = \left(
    \begin{array}{cccc}
      b_{11} & b_{12} & \ldots & b_{1n} \\
      b_{21} & b_{22} & \ldots & b_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      b_{m1} & b_{m2} & \ldots & b_{mn}
    \end{array}
    \right)
$$
とすると、行列の足し算$A+B$は
$$
  A+B = \left(
    \begin{array}{cccc}
      a_{11}+b_{11} & a_{12}+b_{12} & \ldots & a_{1n}+b_{1n} \\
      a_{21}+b_{21} & a_{22}+b_{22} & \ldots & a_{2n}+b_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      a_{m1}+b_{m1} & a_{m2}+b_{m2} & \ldots & a_{mn}+b_{mn} \\
    \end{array}
  \right)
$$
行列の引き算$A-B$は
$$
  A-B = \left(
    \begin{array}{cccc}
      a_{11}-b_{11} & a_{12}-b_{12} & \ldots & a_{1n}-b_{1n} \\
      a_{21}-b_{21} & a_{22}-b_{22} & \ldots & a_{2n}-b_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      a_{m1}-b_{m1} & a_{m2}-b_{m2} & \ldots & a_{mn}-b_{mn} \\
    \end{array}
  \right)
$$
です。これらは行列の列数と行数が一致されているときのみ行えます。<br>

次は行列の掛け算です。